In [2]:
import pymongo
import sys
import time
import pandas as pd
from bson.son import SON
import numpy as np
#Mac에서 실행
if sys.platform =="darwin":
    client = pymongo.MongoClient('143.248.156.197')
#window에서 실행
else:
    client = pymongo.MongoClient('localhost')

db=client.research
sillokManIndex=db.sillokManIndex_new
sillokManInfo = db.sillokManInfo
akssillokJoined = db.akssillokJoined

In [10]:
#index by url
manlist=sillokManInfo.distinct('url');len(manlist)

62596

In [12]:
for man in manlist:
    
    mandocs=sillokManInfo.find({'url':man})
    
    #creation of indexdoc with 생년, 몰년, 관력, date, soldate
    indexdoc={}
    indexdoc['생년']=0
    indexdoc['몰년']=0
    career_names=[]
    career_dates=[]
    career_soldates=[]
   
    #inserting every fields and values into indexdoc
    for mandoc in mandocs:
        career_names.append((mandoc.get("관력","")).replace(" ",''))
        career_dates.append(mandoc.get("date",None))
        career_soldates.append(mandoc.get("soldate",None))
        for mandoc_key in mandoc:
            indexdoc[mandoc_key]=mandoc[mandoc_key]
    
    #inserting special keys
    indexdoc['_id']=man
    indexdoc['career']=career_names
    indexdoc['date']=career_dates
    indexdoc['soldate']=career_soldates
    
    #deleting unnecessary keys
    indexdoc.pop('url', None)
    indexdoc.pop('관력', None)
    indexdoc.pop('왕대', None)
    indexdoc.pop('왕력', None)

    try:
        print(indexdoc['이름'])
        sillokManIndex.insert_one(indexdoc)
    except Exception as e:
        print(e)    
        continue

이수철(李壽鐵)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo/popManDetail.do?manId=M_0015039" }
윤섭(尹涉)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo/popManDetail.do?manId=M_0021802" }
신일청(申一淸)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo/popManDetail.do?manId=M_0022311" }
김현규(金顯奎)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo/popManDetail.do?manId=M_0027942" }
이수남(李壽男)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo/popManDetail.do?manId=M_0004593" }
윤담무(尹覃茂)
E11000 duplicate key error collection: research.sillokManIndex_new index: _id_ dup key: { : "http://sillok.history.go.kr/manInfo

In [147]:
#change "" into 'None' in 'career' field
manlist=sillokManIndex.find()
for man in manlist:
    career=[None if i=="" else i for i in man['career']]
    sillokManIndex.update_one({"_id":man['_id']},{"$set":{"career":career}})

In [148]:
#delete \null 이름
list(sillokManIndex.find({"이름":""}))
sillokManIndex.delete_many({"이름":""})

In [150]:
#delete white space in '이름'
manlist=sillokManIndex.find()
for man in manlist:
    if " " in man['이름']:
        sillokManIndex.update_one({"_id":man['_id']},{"$set":{"이름":man['이름'].replace(' ','')}})

In [77]:
def merge(master, slave):

    recordKeys = [i for i in slave.keys()]
    recordDict = dict()
    for recordKey in recordKeys:
        recordDict[recordKey] = slave[recordKey]
    
    recordDict.pop(slave['career'],None)
    recordDict.pop(slave['date'],None)
    recordDict.pop(slave['soldate'],None)

    mcareer=master.get('career',None)
    mdate=master.get('date',None)
    msoldate=master.get('soldate',None)
    
    mcareer.extend(slave['career'])
    mdate.extend(slave['date'])
    msoldate.extend(slave['soldate'])
    
    recordDict['career']=mcareer
    recordDict['date']=mdate
    recordDict['soldate']=msoldate
    
    for recordKey in recordKeys:
        try:
            sillokManIndex.update({"_id": master["_id"]}, {"$set": {recordKey: recordDict[recordKey]}})
        except:
            pass
    sillokManIndex.delete_one({"_id":slave["_id"]})

In [81]:
def mergesillokManIndex():
    manlist = [i for i in sillokManIndex.find()]
    for man in manlist:
        aliases = [i for i in sillokManIndex.find({"이름":man['이름'], "생년":man['생년']})]
        if len(aliases)==2:
            first = aliases[0]
            second = aliases[1]
            len_of_first=len(list(filter(first['career'],None)))
            len_of_first=len(list(filter(second['career'],None)))
            #둘 다 관직에 오른 기록에 없는 경우, 첫번째 사람으로 merge
            if len_of_first ==0 and len_of_second == 0:
                print("case1")
                merge(first, second)
            #둘 중 한 명만 관직에 오른 기록이 있는 경우, 그 사람으로 merge
            elif len_of_first >=1 and len_of_second == 0:
                print("case2")
                merge(first, second)
            elif len_of_first ==0 and len_of_second >= 1:
                print("case4")
                merge(second, first)
            else:
                print("pass")
                pass

    exceptional = [i for i in sillokManIndex.find({"이름":"윤의립(尹義立)"})]
    merge(exceptional[0],exceptional[1])
    exceptional = [i for i in sillokManIndex.find({"이름":"윤저(尹柢)"})]
    merge(exceptional[0],exceptional[1])
    exceptional = [i for i in sillokManIndex.find({"이름":"조영무(趙英茂)"})]
    merge(exceptional[0],exceptional[1])

In [ ]:
mergesillokManIndex()